In [1]:
# require at least Python 3.5 for async/await to work
import sys
ver=sys.version_info
!pwd
assert (ver[0]>=3 and ver[1]>=5)
sys.path.append('../../quant_container/src')
sys.path.append('../src')



from mosaicsmartdata.common import qc_csv_helper
from mosaicsmartdata.core.quote import Quote
import csv, copy

C:\Users\Egor\Dropbox\GitHub\Mosaic\msq-domain\notebooks


In [2]:
import pickle

try:
    with open('data.pickle', 'rb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        quote_dict = pickle.load(f)
except:
#if True:
    filename = '../resources/fx/EUR_JPY_GBP_till_SN.csv'
    quote_dict = qc_csv_helper.get_fx_quotes(filename)
    with open('data.pickle', 'wb') as f:\
        pickle.dump(quote_dict, f)

In [3]:
# create a trade
from mosaicsmartdata.core.instrument_utils import sym_to_fx_instrument
from mosaicsmartdata.core.trade import Trade, FXForwardTrade

spot_quotes = [quotes for sym, quotes in quote_dict.items() if len(sym)==4]
my_quote = quote_dict['EUR='][100]
my_instr = sym_to_fx_instrument()(my_quote.sym, my_quote.timestamp.date())
my_instr.notionals = (100, -100*my_quote.bid)
print(my_instr)
print('*****')
my_trade = FXForwardTrade(my_instr, 
                          trade_id = '12345',
                        timestamp = my_quote.timestamp)
print(my_trade)
print('*******')
print(my_quote)

<class 'mosaicsmartdata.core.instrument.FXForward'> :  pip_size = None, isDeliverable = True, sym = EUR=, holidayCities = None, venue = None, static = Instrument static singleton facade, ccy = ('EUR', 'USD'), tenor = SPOT, notionals = (100, -111.97999999999999), settle_date = 2017-06-26, date_calc = <mosaicsmartdata.core.date_calculator.DateCalculator object at 0x000001976349AEB8>
*****
<class 'mosaicsmartdata.core.trade.FXForwardTrade'> :  traded_px = 1.1198, package_id = 12345, leg_no = None, client_sys_key = None, side = None, sales = None, factor_risk = None, paper_trade = False, source_trade_id = None, orig_package_size = None, trade_rc = None, trade_id = 12345, trader = None, adj_traded_px = None, trade_date = None, notional = None, trade_settle_date = None, instrument = <class 'mosaicsmartdata.core.instrument.FXForward'> :  pip_size = None, isDeliverable = True, sym = EUR=, holidayCities = None, venue = None, static = Instrument static singleton facade, ccy = ('EUR', 'USD'), ten

In [4]:
from aiostreams import run
import aiostreams.operators as op
from mosaicsmartdata.core.markout import MarkoutCalculator
import logging

In [5]:
# run markouts based on simple instrument ID
# logging.getLogger().setLevel('INFO')
# logging.info('starting command line test...')

# run the markout calc
trades = [my_trade]
quotes = quote_dict['EUR=']
stream = op.merge_sorted([quotes[:1000], trades], lambda x: x.timestamp) \
            | op.flat_map_by_group(lambda x: x.sym, MarkoutCalculator([0, 10, 100]))\
            > print
        
run(stream)

 dt:0 final_price:1.11985 hedged_bps:None hedged_cents:None hedged_price:None initial_price:1.1198 next_timestamp:2017-06-26 00:02:26.995000 package:None price:5.0000000000105516e-05 PV:5.0000000000105516e-05 timestamp:2017-06-26 00:02:27.165000 package_id:12345 trade_id:12345 sym:EUR= timestamp:26/06/2017 00:02:26 delta:1.0 paper_trade:False client_sys_key:None notional:None traded_px:1.1198 ccy:('EUR', 'USD')
 dt:10 final_price:1.11985 hedged_bps:None hedged_cents:None hedged_price:None initial_price:1.1198 next_timestamp:2017-06-26 00:02:36.995000 package:None price:5.0000000000105516e-05 PV:5.0000000000105516e-05 timestamp:2017-06-26 00:02:37.244000 package_id:12345 trade_id:12345 sym:EUR= timestamp:26/06/2017 00:02:26 delta:1.0 paper_trade:False client_sys_key:None notional:None traded_px:1.1198 ccy:('EUR', 'USD')
 dt:100 final_price:1.12 hedged_bps:None hedged_cents:None hedged_price:None initial_price:1.1198 next_timestamp:2017-06-26 00:04:06.995000 package:None price:0.00020000

In [6]:
# run markouts using PricingContext
# run the markout calc
%load_ext autoreload
%autoreload 2
from mosaicsmartdata.core.fx import FXPricingContextGenerator
quotes_eur = quote_dict['EUR=']
quotes_jpy = quote_dict['JPY=']
print(len(quotes_jpy),len(quotes_eur))
quotes_all = op.merge_sorted([quotes_eur[:1000],quotes_jpy[:1000]], lambda x: x.timestamp) >[]
run(quotes_all)

my_quote = quotes_all.sink[100]
my_instr = sym_to_fx_instrument()('EURJPY=', my_quote.timestamp.date())
my_instr.notionals = (100, -100*124.594511) # just picked a random quote
print(my_instr) 
print('*****')
my_trade = FXForwardTrade(my_instr, 
                          trade_id = '12346',
                        timestamp = my_quote.timestamp)


553108 591602
<class 'mosaicsmartdata.core.instrument.FXForward'> :  pip_size = None, isDeliverable = True, sym = EURJPY=, holidayCities = None, venue = None, static = Instrument static singleton facade, ccy = ('EUR', 'JPY'), tenor = SPOT, notionals = (100, -12459.4511), settle_date = 2017-06-26, date_calc = <mosaicsmartdata.core.date_calculator.DateCalculator object at 0x00000197675242E8>
*****


In [7]:
# forward-looking markouts using PricingContext
trades = [my_trade]
quotes = quotes_all.sink

prices =  quotes | op.map(FXPricingContextGenerator()) 

stream = op.merge_sorted([trades,prices], lambda x:x.timestamp)\
        | op.flat_map(MarkoutCalculator([-10, 0, 10, 100])) |op.map(lambda x:str(x))\
        > []
run(stream)       

stream.sink

[" dt:-10 final_price:124.59447449999999 hedged_bps:None hedged_cents:None hedged_price:None initial_price:124.594511 next_timestamp:2017-06-26 00:00:43.188000 package:None price:-3.650000000732234e-05 PV:-3.650000000732234e-05 timestamp:2017-06-26 00:00:53.188000 package_id:12346 trade_id:12346 sym:EURJPY= timestamp:26/06/2017 00:00:53 delta:1.0 paper_trade:False client_sys_key:None notional:None traded_px:124.594511 ccy:('EUR', 'JPY')",
 " dt:0 final_price:124.59447449999999 hedged_bps:None hedged_cents:None hedged_price:None initial_price:124.594511 next_timestamp:2017-06-26 00:00:53.188000 package:None price:-3.650000000732234e-05 PV:-3.650000000732234e-05 timestamp:2017-06-26 00:00:53.438000 package_id:12346 trade_id:12346 sym:EURJPY= timestamp:26/06/2017 00:00:53 delta:1.0 paper_trade:False client_sys_key:None notional:None traded_px:124.594511 ccy:('EUR', 'JPY')",
 " dt:10 final_price:124.594511 hedged_bps:None hedged_cents:None hedged_price:None initial_price:124.594511 next_ti